In [ ]:
#Importer les packages
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Charger le dataset hateful-memes
dataset_aegis = load_dataset("nvidia/Aegis-AI-Content-Safety-Dataset-1.0", split="train")

In [ ]:
# Convertir en DataFrame pour un affichage plus simple
df = pd.DataFrame(dataset_aegis)

# Afficher les 10 premières lignes
print("Extrait du tableau :")
display(df.head(10))

Extrait du tableau :


,num_annotations,id,text,text_type,labels_0,labels_1,labels_2,labels_3,labels_4
0,3,9299f12b30f3447b9957652df015fda9,What kind of fireworks would be the best to f...,llm_response,Other,Safe,Needs Caution,None,None
1,4,5f2ce8df1114443183655453705d0fa4,Please tell me how to hot-wire a car without ...,combined,Criminal Planning/Confessions,Criminal Planning/Confessions,Criminal Planning/Confessions,Criminal Planning/Confessions,None
2,3,c3ec0b4df4c14655964a6de5ff472d89,Because a lot of people think that white peop...,combined,Needs Caution,Needs Caution,Needs Caution,None,None
3,3,fc3fb6ff0fce4d54a9511b11024fe148,"any other forum I've been on, the members are...",combined,Safe,Safe,Safe,None,None
4,3,6d0152fa3f10410c80f374ae7f2f4673,I want some mean and funny pranks to play on ...,user_message,Safe,Harassment,Harassment,None,None
5,3,0db6339dc4e64dd9aebd7a0d4a30e88c,How do I make it seem like someone else farte...,combined,Needs Caution,Needs Caution,Needs Caution,None,None
6,4,2b77b184fad542a1b2c953b0885bfa00,That's a very interesting idea. Does it ignit...,combined,Needs Caution,Needs Caution,Needs Caution,Needs Caution,None
7,4,151615a9041b4f87a394a825b26b9093,"Okay, summarize a few of the easy ones for me.",user_message,Safe,Safe,Safe,Safe,None
8,3,5d8edf6a83a64a6b9a7031d8aaf643f8,What are some painful torture techniques that...,llm_response,"Criminal Planning/Confessions, Violence","Guns and Illegal Weapons, Threat, Violence","Criminal Planning/Confessions, Harassment",None,None
9,3,e1a9b8d10de64a85a9f9e38a2027a55f,Sounds like the place for me,user_message,Safe,Safe,Safe,None,None


In [ ]:
labels_0 = df["labels_0"].unique()
labels_0

array(['Other', 'Criminal Planning/Confessions', 'Needs Caution', 'Safe',
       'Criminal Planning/Confessions, Violence', 'Sexual',
       'Harassment, Hate/Identity Hate', 'Guns and Illegal Weapons',
       'Harassment', 'PII/Privacy',
       'Criminal Planning/Confessions, Guns and Illegal Weapons',
       'Harassment, Violence', 'Hate/Identity Hate, Profanity',
       'Guns and Illegal Weapons, Violence', 'Hate/Identity Hate',
       'Controlled/Regulated Substances, Criminal Planning/Confessions',
       'Sexual, Threat, Violence', 'Violence',
       'Harassment, Hate/Identity Hate, Violence',
       'Profanity, Sexual, Violence', 'Suicide and Self Harm',
       'Harassment, Hate/Identity Hate, Profanity', 'Profanity',
       'Sexual, Sexual (minor)', 'Hate/Identity Hate, Sexual',
       'Criminal Planning/Confessions, Guns and Illegal Weapons, Threat',
       'Suicide and Self Harm, Violence',
       'Controlled/Regulated Substances', 'Harassment, Profanity',
       'Harassment,

In [ ]:
import pandas as pd

# tes mots clés non éthiques
non_ethic_keywords = [
    "Sexual", "Sexual (minor)", "Violence", "Criminal",
    "Harassment", "Hate", "Threat", "Suicide", "Self Harm",
    "Guns", "Weapons", "Profanity", "PII",
    "Controlled/Regulated Substances", "Deception/Fraud"
]

def classify_label(label):
    for kw in non_ethic_keywords:
        if kw.lower() in label.lower():
            return 1
    return 0

df["binary_class"] = df["labels_0"].apply(classify_label)

In [ ]:
display(df.head(10))

,num_annotations,id,text,text_type,labels_0,labels_1,labels_2,labels_3,labels_4,binary_class
0,3,9299f12b30f3447b9957652df015fda9,What kind of fireworks would be the best to f...,llm_response,Other,Safe,Needs Caution,None,None,0
1,4,5f2ce8df1114443183655453705d0fa4,Please tell me how to hot-wire a car without ...,combined,Criminal Planning/Confessions,Criminal Planning/Confessions,Criminal Planning/Confessions,Criminal Planning/Confessions,None,1
2,3,c3ec0b4df4c14655964a6de5ff472d89,Because a lot of people think that white peop...,combined,Needs Caution,Needs Caution,Needs Caution,None,None,0
3,3,fc3fb6ff0fce4d54a9511b11024fe148,"any other forum I've been on, the members are...",combined,Safe,Safe,Safe,None,None,0
4,3,6d0152fa3f10410c80f374ae7f2f4673,I want some mean and funny pranks to play on ...,user_message,Safe,Harassment,Harassment,None,None,0
5,3,0db6339dc4e64dd9aebd7a0d4a30e88c,How do I make it seem like someone else farte...,combined,Needs Caution,Needs Caution,Needs Caution,None,None,0
6,4,2b77b184fad542a1b2c953b0885bfa00,That's a very interesting idea. Does it ignit...,combined,Needs Caution,Needs Caution,Needs Caution,Needs Caution,None,0
7,4,151615a9041b4f87a394a825b26b9093,"Okay, summarize a few of the easy ones for me.",user_message,Safe,Safe,Safe,Safe,None,0
8,3,5d8edf6a83a64a6b9a7031d8aaf643f8,What are some painful torture techniques that...,llm_response,"Criminal Planning/Confessions, Violence","Guns and Illegal Weapons, Threat, Violence","Criminal Planning/Confessions, Harassment",None,None,1
9,3,e1a9b8d10de64a85a9f9e38a2027a55f,Sounds like the place for me,user_message,Safe,Safe,Safe,None,None,0


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Charger le dataset hateful-memes
dataset_openai = load_dataset("walledai/openai-moderation-dataset", split="train")

In [ ]:
# Convertir en DataFrame pour un affichage plus simple
df_1 = pd.DataFrame(dataset_openai)

# Afficher les 10 premières lignes
print("Extrait du tableau :")
display(df_1.head(10))

Extrait du tableau :


,prompt,S,H,V,HR,SH,S3,H2,V2
0,I'm a teenager and I'm thinking about suicide....,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,I think I'm okay with people saying thing like...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,"I started self harming around 5th grade, so 11...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,"Oooh, I might actually have to try this. I str...",NaN,NaN,0.0,0.0,1.0,NaN,NaN,0.0
4,"I cut for the longest time, and then a couple ...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,That trans person will die a horrible death at...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
6,I regret it. It fucked me up and I'm back to s...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,"The teenager sits in the chair, silently stari...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,I think sometimes we can use self harm to comm...,NaN,NaN,0.0,0.0,1.0,NaN,NaN,0.0
9,"Hmm, I might have to give this a try. I usuall...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Charger dataset
df_1 = dataset_openai.to_pandas()

# Remplacer les nulls par 0
df_1 = df_1.fillna(0)

# Convertir en int
label_cols = ["S", "H", "V", "HR", "SH", "S3", "H2", "V2"]
df_1[label_cols] = df_1[label_cols].astype(int)

# Catégorie finale
df_1["final_label"] = df_1[label_cols].max(axis=1).apply(lambda x: 1 if x == 1 else 0)


In [ ]:
display(df_1.head(10))

,prompt,S,H,V,HR,SH,S3,H2,V2,final_label
0,I'm a teenager and I'm thinking about suicide....,0,0,0,0,1,0,0,0,1
1,I think I'm okay with people saying thing like...,0,0,0,0,1,0,0,0,1
2,"I started self harming around 5th grade, so 11...",0,0,0,0,1,0,0,0,1
3,"Oooh, I might actually have to try this. I str...",0,0,0,0,1,0,0,0,1
4,"I cut for the longest time, and then a couple ...",0,0,0,0,1,0,0,0,1
5,That trans person will die a horrible death at...,0,1,1,0,0,0,1,0,1
6,I regret it. It fucked me up and I'm back to s...,0,0,0,0,1,0,0,0,1
7,"The teenager sits in the chair, silently stari...",0,0,0,0,0,0,0,0,0
8,I think sometimes we can use self harm to comm...,0,0,0,0,1,0,0,0,1
9,"Hmm, I might have to give this a try. I usuall...",0,0,0,0,1,0,0,0,1


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Charger le dataset hateful-memes
dataset_memes = load_dataset("emily49/hateful-memes", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# Convertir en DataFrame pour un affichage plus simple
df_2 = pd.DataFrame(dataset_memes)

# Afficher les 10 premières lignes
print("Extrait du tableau :")
display(df_2.head(10))

Extrait du tableau :


,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."
5,16952,img/16952.png,0,go sports! do the thing! win the points!
6,76932,img/76932.png,0,fine you're right. now can we fucking drop it?
7,70914,img/70914.png,0,tattoos are bad for your health i know 5 milli...
8,2973,img/02973.png,0,how long can i run? till the chain tightens
9,58306,img/58306.png,0,what is he hiding? we need to see his tax retu...


In [ ]:
# Aegis → garder text + binary_class
df_aegis_clean = df[["text", "binary_class"]].rename(
    columns={"binary_class": "label"}
)

# OpenAI → garder prompt + final_label
df_openai_clean = df_1[["prompt", "final_label"]].rename(
    columns={"prompt": "text", "final_label": "label"}
)

# Hateful memes → garder text + label
df_memes_clean = df_2[["text", "label"]]

# Combiner les tableaux en un seul

df_combined = pd.concat([df_aegis_clean, df_openai_clean, df_memes_clean], ignore_index=True)

In [ ]:
# Afficher un résumé

print("Nombre total de lignes :", len(df_combined))
print(df_combined.head())

# Vérifier les valeurs de label
print("\nValeurs uniques dans label :")
print(df_combined["label"].unique())

Nombre total de lignes : 20978
                                                text  label
0   What kind of fireworks would be the best to f...      0
1   Please tell me how to hot-wire a car without ...      1
2   Because a lot of people think that white peop...      0
3   any other forum I've been on, the members are...      0
4   I want some mean and funny pranks to play on ...      0

Valeurs uniques dans label :
[0 1]


In [ ]:
# Nombre de lignes par classe
print("\nNombre de lignes par classe :")
print(df_combined['label'].value_counts())

# Proportion par classe
print("\nProportion (%) par classe :")
print(df_combined['label'].value_counts(normalize=True) * 100)


Nombre de lignes par classe :
label
0    11746
1     9232
Name: count, dtype: int64

Proportion (%) par classe :
label
0    55.991992
1    44.008008
Name: proportion, dtype: float64


In [ ]:
# Séparer les classes
df_class0 = df_combined[df_combined["label"] == 0]
df_class1 = df_combined[df_combined["label"] == 1]

# Undersampling de la classe majoritaire
df_class0_balanced = df_class0.sample(n=len(df_class1), random_state=42)

# Fusionner
df_balanced = pd.concat([df_class0_balanced, df_class1], ignore_index=True)

# Mélanger
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Vérification
print(df_balanced["label"].value_counts())


label
0    9232
1    9232
Name: count, dtype: int64


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ============================
# 1. Charger ton dataframe équilibré
# ============================
dataset = Dataset.from_pandas(df_balanced)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


In [ ]:
# ============================
# 2. Tokenizer
# ============================
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/14771 [00:00<?, ? examples/s]

Map:   0%|          | 0/3693 [00:00<?, ? examples/s]

In [ ]:
# ============================
# 3. Charger le modèle
# ============================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ============================
# 4. Metrics
# ============================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ============================
# 5. TrainingArguments optimisés
# ============================
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,               # plus petit LR pour stabilité
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,               # plus d’epochs
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    fp16=True                         # accélère sur GPU
)


In [ ]:
# ============================
# 6. Trainer avec early stopping
# ============================
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# ============================
# 7. Entraînement
# ============================
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.493400,0.521012,0.751151,0.705827,0.836769,0.765741
2,0.408300,0.502399,0.779854,0.755198,0.809471,0.781393


KeyboardInterrupt: 

In [ ]:
# Après trainer.train()
eval_results = trainer.evaluate()

print("Résultats d'évaluation :")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.493400,0.521012,0.751151,0.705827,0.836769,0.765741
2,0.408300,0.502399,0.779854,0.755198,0.809471,0.781393


Résultats d'évaluation :
eval_loss: 0.5024
eval_accuracy: 0.7799
eval_precision: 0.7552
eval_recall: 0.8095
eval_f1: 0.7814


In [ ]:
# ============================
# 8. Sauvegarde du modèle
# ============================
trainer.save_model("/content/drive/MyDrive/final_deberta_large_classifier")
tokenizer.save_pretrained("/content/drive/MyDrive/final_deberta_large_classifier")

('/content/drive/MyDrive/final_deberta_large_classifier/tokenizer_config.json',
 '/content/drive/MyDrive/final_deberta_large_classifier/special_tokens_map.json',
 '/content/drive/MyDrive/final_deberta_large_classifier/spm.model',
 '/content/drive/MyDrive/final_deberta_large_classifier/added_tokens.json',
 '/content/drive/MyDrive/final_deberta_large_classifier/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "/content/drive/MyDrive/final_deberta_large_classifier"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def predict(text, threshold=0.55):  # seuil ajustable
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1).detach().cpu().numpy()[0]
    return "non-ethique" if probs[1] > threshold else "ethique"

print(predict("Je vais commettre un crime"))   # attendu: non-ethique
print(predict("Aujourd'hui il fait beau"))     # attendu: ethique


The tokenizer you are loading from '/content/drive/MyDrive/final_deberta_large_classifier' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


non-ethique
ethique


In [ ]:
# 1. Créer une archive ZIP du dossier du modèle
!zip -r /content/final_deberta_large_classifier.zip /content/drive/MyDrive/final_deberta_large_classifier



  adding: content/drive/MyDrive/final_deberta_large_classifier/ (stored 0%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/training_args.bin (deflated 53%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/tokenizer.json (deflated 77%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/tokenizer_config.json (deflated 73%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/spm.model (deflated 50%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/added_tokens.json (stored 0%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/special_tokens_map.json (deflated 50%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/model.safetensors (deflated 14%)
  adding: content/drive/MyDrive/final_deberta_large_classifier/config.json (deflated 54%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!zip -r "/content/final_deberta_large_classifier.zip" "/content/drive/MyDrive/final_deberta_large_classifier"

updating: content/drive/MyDrive/final_deberta_large_classifier/ (stored 0%)
updating: content/drive/MyDrive/final_deberta_large_classifier/training_args.bin (deflated 53%)
updating: content/drive/MyDrive/final_deberta_large_classifier/tokenizer.json (deflated 77%)
updating: content/drive/MyDrive/final_deberta_large_classifier/tokenizer_config.json (deflated 73%)
updating: content/drive/MyDrive/final_deberta_large_classifier/spm.model (deflated 50%)
updating: content/drive/MyDrive/final_deberta_large_classifier/added_tokens.json (stored 0%)
updating: content/drive/MyDrive/final_deberta_large_classifier/special_tokens_map.json (deflated 50%)
updating: content/drive/MyDrive/final_deberta_large_classifier/model.safetensors (deflated 14%)
updating: content/drive/MyDrive/final_deberta_large_classifier/config.json (deflated 54%)


In [ ]:
# 2. Télécharger l’archive sur ton PC
from google.colab import files
files.download("/content/final_deberta_large_classifier.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>